In [1]:
import gc
from time import time

from pathlib import Path
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pandarallel import pandarallel
import warnings
import _read_data_files_helper as data_files

warnings.filterwarnings('ignore')

# setup pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=6)
tqdm.pandas(desc='Negative samples!')

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
path_model_data = Path("")
path_base = Path("") #original data/csv files
path_s3 = "CF_model_input_S3"
path_s1 = 'CF_model_input_neucf_S1'
path_s2 = 'CF_model_input_neucf_S2'
number_negative_eg = 30

### Test number of negatives per week

In [ ]:
file_path = list((path_model_data / path_s3).rglob('*.parquet'))

for file in file_path:
    df_test = pd.read_parquet(file)
    negative_count = df_test[df_test.purchase==0]
    negative_count = negative_count.groupby(['customer_id'], as_index=False).article_id.nunique()
    assert (negative_count.article_id !=30).sum() == 0

## Negative strategy 1 & 2

In [13]:
filepath_tr = list((path_model_data / path_s1).glob('train_*.parquet'))[0]
filepath_val = list((path_model_data / path_s1).glob('val_*.parquet'))[0]
stg_one_train = pd.read_parquet(filepath_tr)
stg_one_val = pd.read_parquet(filepath_val)

In [17]:
val_custs = stg_one_val.customer_id.unique()
train_custs = stg_one_train.customer_id.unique()
mask = np.isin(val_custs, train_custs)
val_custs[~mask].shape[0] == 0

True

In [18]:
filepath_tr = list((path_model_data / path_s2).glob('train_*.parquet'))[0]
filepath_val = list((path_model_data / path_s2).glob('val_*.parquet'))[0]
stg_one_train = pd.read_parquet(filepath_tr)
stg_one_val = pd.read_parquet(filepath_val)

In [19]:
val_custs = stg_one_val.customer_id.unique()
train_custs = stg_one_train.customer_id.unique()
mask = np.isin(val_custs, train_custs)
val_custs[~mask].shape[0] == 0

True